In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from functools import partial
import matplotlib.pyplot as plt

In [ ]:
csv_path = '/content/drive/My Drive/project/mango/test/'
image_path = '/content/drive/My Drive/project/mango/test/image/Test/'

In [ ]:
def load_csv(csv_path, file_name):
    dataset = pd.read_csv(os.path.join(csv_path, file_name), low_memory = False)
    # dataset = pd.read_csv(os.path.join(csv_path, file_name), header = None, low_memory = False)
    dataset.fillna('1', inplace = True)
    dataset_array = np.array(dataset)
    return dataset, dataset_array

In [ ]:
dataset, dataset_array = load_csv(csv_path, 'Test_mangoXYWH.csv')

# Write TFRecord Dataset

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list = tf.train.BytesList(value = [value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))


def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

def segment(data):
    feature_set= []
    img_name = []
    demension = len(data[0])
    segment = list(range(1, demension, 5))
    for i in range(len(data)):
        for seg in segment:
            if data[i][seg] == '1':
                pass
            else:
                sub_data = data[i][seg:seg + 5]
                img = data[i][0]
                feature_set.append(sub_data)
                img_name.append(img)
    img_name = np.array(img_name)
    feature_set = np.array(feature_set)
    label = feature_set[:,4]
    w_h = feature_set[:,:4].astype('float32')
    return img_name, feature_set, label, w_h


def image_example(image, b_box):
    # image = plt.imread(os.path.join(image_path, file_name))
    # image_string = open(os.path.join(image_path, file_name), 'rb').read()
    # image_shape = tf.image.decode_jpeg(image_string).shape
    image_string = tf.image.encode_jpeg(image)
    feature = {
      'image_raw': _bytes_feature(image_string),
      'target':_bytes_feature(b_box),
    }
    return tf.train.Example(features = tf.train.Features(feature = feature))

def image_example_2(image, cordinate):
    image_string = tf.image.encode_jpeg(image)
    feature = {
      'image_raw': _bytes_feature(image_string),
      'cordinate' : _bytes_feature(cordinate)
    }
    return tf.train.Example(features = tf.train.Features(feature = feature))
def y_label(label):
  target = np.zeros((5,4), dtype = 'float32')
  if label == '不良-乳汁吸附':
    o_h_e[0,0] = 1
  elif label == '不良-機械傷害':
    o_h_e[0,1] = 1
  elif label == '不良-炭疽病':
    o_h_e[0,2] = 1
  elif label == '不良-著色不佳':
    o_h_e[0,3] = 1
  else:
    o_h_e[0,4] = 1
  # return o_h_e.tostring()
  return o_h_e.tobytes()
  # return o_h_e

def scale_bbox(cor, x_scale, y_scale):
  x, y, w, h = cor
  new_x, new_y, new_w, new_h = int(np.round(x * x_scale)), int(np.round(y * y_scale)), \
                               int(np.round(w * x_scale)), int(np.round(h * y_scale))
  return np.array([new_x / 224, new_y / 224, new_w / 224, new_h / 224]).astype(np.float32)

In [ ]:
cor = dataset_array[:, 1:]

In [ ]:
a = scale_bbox(cor[0], 0.5, 0.4)
cor[0]

array([189, 64, 1078, 793], dtype=object)

In [ ]:
print(a)
print(tf.io.decode_raw(a.tobytes(), tf.float32))

[0.41964287 0.11607143 2.40625    1.4151785 ]
tf.Tensor([0.41964287 0.11607143 2.40625    1.4151785 ], shape=(4,), dtype=float32)


In [ ]:
cor = dataset_array[:, 1:].astype(int)
with tf.io.TFRecordWriter(os.path.join(csv_path, 'test_cor.tfrecord')) as writer:
  a, b = 0, 0
  for i in range(len(dataset_array)):
    img_name = dataset_array[i, 0]
    if os.path.isfile(image_path + img_name):
      w_image = plt.imread(os.path.join(image_path, img_name))
      x_scale, y_scale = (224 / w_image.shape[1]), (224 / w_image.shape[0])
      w_image = cv2.resize(w_image, (224, 224))
      new_box = scale_bbox(cor[i], x_scale, y_scale)
      new_box = new_box.tobytes()
      tf_example = image_example(w_image, new_box)
      writer.write(tf_example.SerializeToString())
      b += 1
    a += 1
writer.close()
print(a, b)

7363 7363


#Read TF Dataset

In [ ]:
BATCH_SIZE = 100
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = image/255
    # image = tf.reshape(image, [224, 224, 3])
    return image

def decode_label(label):
  label = tf.io.decode_raw(label, tf.float32)
  # label = p
  # label = tf.cast(label, tf.float32)
  return label

def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            'image_raw': tf.io.FixedLenFeature([], tf.string),
            'target': tf.io.FixedLenFeature([], tf.string),
        }
        if labeled
        else {"image_raw": tf.io.FixedLenFeature([], tf.string)}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image_raw'])
    if labeled:
        label = decode_label(example['target'])
        # label = tf.cast(label, tf.int32)
        return image, label
    return image

def load_dataset(filenames, labeled = False):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = True  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)  
# uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled = labeled), num_parallel_calls = AUTOTUNE)
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames, labeled = False):
    dataset = load_dataset(filenames, labeled = labeled)
#     dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size = AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
data = get_dataset(os.path.join(csv_path, 'test_cor.tfrecord'), labeled = True)

In [ ]:
image_batch, label_batch = next(iter(data))